<a href="https://colab.research.google.com/github/LucaNavarra/MC3/blob/main/MC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerie utilizzate #

In [ ]:
import scipy.io as sio
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import random
from random import shuffle
import matplotlib.pyplot as plt

#Check device#

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#rendere l'esecuzione deterministica
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#Classe per lettura dati#

In [ ]:
class My_DatasetFolder(Dataset):
  def __init__(self, root,  transform, is_valid_file, list_classes):
    self.root = root     #path dei dati
    self.transform = transform    #lista di trasformazioni
    self.is_valid_file = is_valid_file #check per capire quali file leggere
    self.list_classes = list_classes   #lista di classi
    self.samples = self.__get_samples()

  def __len__(self):
    return len(self.samples)

  def __get_samples(self):
    ListFiles=[]
    for c in self.list_classes:
      list_folder = os.listdir(self.root + '/' + c)
      for folder in list_folder:
        listofFiles = os.listdir(self.root + '/' + c + '/' + folder)
        for file in listofFiles:
          if self.is_valid_file(self.root + '/' + c + '/' + folder + '/' + file):
            ListFiles.append((self.root + '/' + c + '/' + folder + '/' + file, self.list_classes.index(c)))

    return ListFiles

  def loader_fc(self, path):
    img = np.load(path)
    return img

  def __getitem__(self, index: int):
    path, target = self.samples[index]
    sample = self.loader_fc(path)
    if self.transform is not None:
      sample = self.transform(sample)
    return sample, target


'''
un esempio di utilizzo
is_valid_file = lambda path: path.endswith('.npy')

My_DatasetFolder(root = MyPath,  transform = data_transforms, is_valid_file = is_valid_file, list_classes = ['benign', 'malignant'])
'''

#Estrazione di features sul train

In [ ]:
outfeatures_train =[]

data_transforms = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Resize(224),   #questo è solo un esempio!
                                      ])

'''
Inserire il modello che si vuole considerare
'''

model = ....
model.eval()

class_list_train = []
class_list_test = []
classes = ['benign', 'malignant']
data_dir = 'data'   #path dei dati

for c in classes:
  folder = os.listdir(data_dir + '/train/' + c)
  for f in folder:
    listImage = os.listdir(data_dir + '/train/' + c + '/' + f)
    for input in listImage:
      img = np.load(data_dir + '/train/' + c + '/' + f + '/' + input)
      img = data_transforms(img).type(torch.FloatTensor).to(device)
      img = img.unsqueeze_(0)
      outputs = model(img)
      outfeatures_train.append((np.squeeze(outputs.cpu().detach()).numpy()))
      class_list_train.append(classes.index(c))